In [1]:
%load_ext lab_black
%load_ext watermark
%matplotlib inline

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymc3 as pm
import scipy as sp
import seaborn as sns
import theano.tensor as tt
import warnings

from scipy.special import expit as logistic
from scipy.special import softmax

warnings.simplefilter(action="ignore", category=FutureWarning)
az.style.use("arviz-darkgrid")
RANDOM_SEED = 8927
np.random.seed(286)

In [5]:
d = pd.read_excel("data/results_by_arrdmt.xlsx", index_col=0)
d

,date,ville,arrondissement,N,farleft,left,green,center,right,farright,other
0,2008-05-23,Paris,1,6127,75.0,2289,439,531.0,2641,152,0
1,2014-03-25,Paris,1,6079,154.0,1663,659,NaN,3144,306,153
2,2008-05-23,Paris,2,6736,90.0,2231,2016,621.0,1543,167,68
3,2014-03-25,Paris,2,6857,192.0,1565,2260,NaN,1663,272,905
4,2008-05-23,Paris,3,11974,225.0,6685,1237,1111.0,2458,258,0
5,2014-03-25,Paris,3,11615,464.0,5493,1252,NaN,3379,580,447
6,2008-05-23,Paris,4,10573,151.0,5127,834,863.0,3312,286,0
7,2014-03-25,Paris,4,9850,376.0,3684,916,NaN,3725,512,637
8,2008-05-23,Paris,5,23614,688.0,8187,1287,3385.0,8958,418,691
9,2014-03-25,Paris,5,22243,976.0,7550,1985,NaN,6336,805,4591
